<a href="https://colab.research.google.com/github/kevinMGII/Grupo7-Practica1/blob/main/Grupo7_Practica1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **PRIMERA PRÁCTICA**.
Predicción del Abandono de Empleados

 ---

Carga y visualización de los datos desde Google Colab.

In [6]:
import pandas as pd

data_train = pd.read_csv("./attrition_availabledata_12.csv.gz")                 # Cargamos el conjunto de datos de entrenamiento
data_test = pd.read_csv("./attrition_competition_12.csv.gz")                    # Cargamos el conjunto de datos de test utilizando el almacenamiento local de Google Colab.

data_train.head()                                                               # Mostramos las 5 primeras filas del conjunto de datos de entrenamiento. (Not Working)
data_test.head()                                                                # Mostramos las 5 primeras filas del conjunto de datos de test.

,hrs,absences,JobInvolvement,PerformanceRating,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,Age,BusinessTravel,Department,...,NumCompaniesWorked,Over18,PercentSalaryHike,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager
0,6.894187,6,2,3,2.0,4.0,2.0,34,Travel_Rarely,Sales,...,0.0,Y,12,8,1,16.0,5,15,0,9
1,7.684525,7,2,3,3.0,4.0,1.0,30,Non-Travel,Research & Development,...,4.0,Y,18,8,3,9.0,2,5,1,4
2,6.806975,9,3,3,2.0,4.0,2.0,29,Travel_Frequently,Research & Development,...,1.0,Y,11,8,1,10.0,2,10,0,9
3,6.028659,6,3,3,2.0,1.0,3.0,44,Travel_Rarely,Research & Development,...,9.0,Y,15,8,0,24.0,3,20,3,6
4,7.618041,12,2,3,3.0,4.0,3.0,28,Non-Travel,Research & Development,...,0.0,Y,13,8,0,10.0,2,9,7,5


En el desarrollo de esta práctica se utilizarán los archivos que contienen los datos con la extension **12**. Cumpliendo así las consideraciones generales, siendo la suma de los últimos dos dígitos nuestros NIA's 12 en ambos casos.

 ---

**EDA SIMPLIFICADO** (puedes cambiar o borrar lo que quierasssssssssssssssss) me tengo que ir otra vez pero luego vuelvo

Cargamos el archivo *attrition_availabledata_00* para hacer el EDA inicial.

In [3]:
import pandas as pd
#import numpy as np
#import matplotlib.pyplot as plt
#import seaborn as sns

file_route = route_datasets + "attrition_availabledata_00.csv.gz"
data = pd.read_csv(file_route)

data.head()



FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/My Drive/attrition_datasets/train_test/attrition_availabledata_00.csv.gz'

Número de  variables e instancias

In [ ]:
num_instancias, num_variables = data.shape
print(f"Número de instancias: {num_instancias}")
print(f"Número de variables: {num_variables}")

Número de instancias: 2940
Número de variables: 31


Tipos de variables (categóricas, numéricas u ordinales)

In [ ]:
# Tipos de datos de cada columna
print(data.dtypes)

# Separamos variables en categóricas y numéricas
categorical_vars = data.select_dtypes(include=['object', 'category']).columns
numeric_vars = data.select_dtypes(include=['int64', 'float64']).columns

print(f"Variables categóricas: {categorical_vars}")
print(f"Variables numéricas: {numeric_vars}")
# Las variables ordinales deben identificarse manualmente

hrs                        float64
absences                     int64
JobInvolvement               int64
PerformanceRating            int64
EnvironmentSatisfaction    float64
JobSatisfaction            float64
WorkLifeBalance            float64
Age                          int64
BusinessTravel              object
Department                  object
DistanceFromHome             int64
Education                    int64
EducationField              object
EmployeeCount                int64
EmployeeID                   int64
Gender                      object
JobLevel                     int64
JobRole                     object
MaritalStatus               object
MonthlyIncome                int64
NumCompaniesWorked         float64
Over18                      object
PercentSalaryHike            int64
StandardHours                int64
StockOptionLevel             int64
TotalWorkingYears          float64
TrainingTimesLastYear        int64
YearsAtCompany               int64
YearsSinceLastPromot